## Padrón Comunidad de Madrid

2.Crear otro notebook donde leyendo el fichero original en CSV obtener algunas métricas.

In [19]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV to Dataset").getOrCreate()

In [20]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType() \
    .add('COD_DISTRITO', IntegerType(), True) \
    .add('DESC_DISTRITO', StringType(), True) \
    .add('COD_DIST_BARRIO', IntegerType(), True) \
    .add('DESC_BARRIO', StringType(), True) \
    .add('COD_BARRIO', IntegerType(), True) \
    .add('COD_DIST_SECCION', IntegerType(), True) \
    .add('COD_SECCION', IntegerType(), True) \
    .add('COD_EDAD_INT', IntegerType(), True) \
    .add('EspanolesHombres', IntegerType(), True) \
    .add('EspanolesMujeres', IntegerType(), True) \
    .add('ExtranjerosHombres', IntegerType(), True) \
    .add('ExtranjerosMujeres', IntegerType(), True)

df_schema = spark.read.format('csv') \
    .option('header',True) \
    .option('delimiter', ';') \
    .schema(schema) \
    .load('padron_madrid/Rango_Edades_Seccion_202011.csv')

df_schema.printSchema()
df_schema.show(5)


root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+-

In [21]:
#Cuantas filas tiene el dataframe
df_schema.count()

237593

In [22]:
#Importar librería para utilizar las funciones
from pyspark.sql import functions as pyf

In [23]:
#Sustituir valores nulos por 0
df_schema_Null0 = df_schema.na.fill(0)

In [24]:
#Promedios por dsitrito, ordenados alfabeticamente por columna distrito

df_group = df_schema_Null0.groupBy('DESC_DISTRITO') \
    .avg('EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres') \
    .orderBy('DESC_DISTRITO')
df_group.show()


+--------------------+---------------------+---------------------+-----------------------+-----------------------+
|       DESC_DISTRITO|avg(EspanolesHombres)|avg(EspanolesMujeres)|avg(ExtranjerosHombres)|avg(ExtranjerosMujeres)|
+--------------------+---------------------+---------------------+-----------------------+-----------------------+
|ARGANZUELA          |    6.072125304820859|    6.953198274244982|     0.7358844494466329|     0.8274244982179704|
|BARAJAS             |      7.3536503683858|    7.763228399196249|     0.7803081044876088|     0.9561286001339585|
|CARABANCHEL         |    5.407337454212454|    6.247424450549451|     1.5400641025641026|      1.711080586080586|
|CENTRO              |    4.915913958469131|   4.9248533382996555|     1.7890865071235682|     1.5495856224974391|
|CHAMARTIN           |    6.020822905195716|   7.1854039443387725|     0.6085694263690059|     0.8993893282610872|
|CHAMBERI            |    4.476936966513461|    5.592662508207485|     0.6060407

In [25]:
#Creamos un DF solo con los datos numericos

df_num = df_schema_Null0.select('EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres')
df_num.show()

+----------------+----------------+------------------+------------------+
|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+----------------+----------------+------------------+------------------+
|               5|               0|                 1|                 0|
|               2|               3|                 0|                 5|
|               6|               1|                 1|                 0|
|               2|               0|                 0|                 3|
|               3|               3|                 0|                 0|
|               2|               2|                 2|                 1|
|               2|               1|                 0|                 0|
|               2|               3|                 1|                 1|
|               4|               0|                 0|                 1|
|               2|               4|                 0|                 0|
|               5|               1|   

In [26]:
#Otras agregaciones por distrito

df_schema_Null0.groupBy('DESC_DISTRITO') \
    .agg(pyf.max('EspanolesHombres'), pyf.stddev('EspanolesHombres'), \
        pyf.max('EspanolesMujeres'), pyf.stddev('EspanolesMujeres')) \
    .orderBy('DESC_DISTRITO') \
    .show()

+--------------------+---------------------+-----------------------------+---------------------+-----------------------------+
|       DESC_DISTRITO|max(EspanolesHombres)|stddev_samp(EspanolesHombres)|max(EspanolesMujeres)|stddev_samp(EspanolesMujeres)|
+--------------------+---------------------+-----------------------------+---------------------+-----------------------------+
|ARGANZUELA          |                   43|            4.969822973168646|                   40|             5.16527013475927|
|BARAJAS             |                   48|            6.158284725266348|                   48|            6.433101231132147|
|CARABANCHEL         |                   56|            4.592413601061038|                   72|            4.789073908318246|
|CENTRO              |                   30|             4.26672937275971|                   24|           3.5767189286107928|
|CHAMARTIN           |                   28|           4.1273731518404535|                   37|            4.3

In [27]:
#Mostar el total personas por distrito, ordenado por distrito y renombrar las columnas nuevas

df_total = df_schema_Null0.groupBy('DESC_DISTRITO') \
    .sum('EspanolesHombres', 'EspanolesMujeres', 'ExtranjerosHombres', 'ExtranjerosMujeres') \
    .withColumnRenamed('sum(EspanolesHombres)', 'Total_EspanolesHombres') \
    .withColumnRenamed('sum(EspanolesMujeres)', 'Total_EspanolesMujeres') \
    .withColumnRenamed('sum(ExtranjerosHombres)', 'Total_ExtranjerosHombres') \
    .withColumnRenamed('sum(ExtranjerosMujeres)', 'Total_ExtranjerosMujeres') \
    .orderBy('DESC_DISTRITO')

df_total.show()


+--------------------+----------------------+----------------------+------------------------+------------------------+
|       DESC_DISTRITO|Total_EspanolesHombres|Total_EspanolesMujeres|Total_ExtranjerosHombres|Total_ExtranjerosMujeres|
+--------------------+----------------------+----------------------+------------------------+------------------------+
|ARGANZUELA          |                 64741|                 74135|                    7846|                    8822|
|BARAJAS             |                 21958|                 23181|                    2330|                    2855|
|CARABANCHEL         |                 94477|                109155|                   26908|                   29896|
|CENTRO              |                 52792|                 52888|                   19213|                   16641|
|CHAMARTIN           |                 60142|                 71775|                    6079|                    8984|
|CHAMBERI            |                 54547|   

In [35]:
#Filtrar los datos por distrito Carabanchel

#df_total2 = df_total.filter(pyf.col('DESC_DISTRITO') == 'CARABANCHEL')
#df_total2.show()

### Conclusiones:

.- El distrito con mas población es Carabanchel. Tmabién es el que tiene mayor numero de españoles, frente a Barajas que es el que menos.

.- En el distrito de Villa de Vallecas es donde viven de promedio mayor numero mujeres españolas frente a Moratalez que es el que menos tiene.

In [16]:
spark.stop()